# Magliette AzIM: ordini, prezzi e consegne

L'**Azienda degli Ingegneri Matematici** ha bisogno del tuo aiuto: uno stagista chiamato F.B. ha eliminato il database degli ordini delle stupende **magliette ufficiali AzIM**! Tutto quello che rimane sono 420 frammenti di email con le richieste dei clienti, spetta a te recuperarle e determinare che ordini accettare!


0. Nella cartella data trovi tutti i file di testo con i frammenti di email, sono tutti diversi! Contengono la **taglia** delle magliette, la **quantità** di magliette ordinate, la **posizione** in coordinate GCS della consegna, il **nome** del cliente e il **prezzo** a cui vorrebbe acquistare il singolo capo


1. [1pt] Registra gli ordini in una **lista di oggetti** "Order"


2. [1pt] Ogni cliente ha richiesto un prezzo diverso, ma il prezzo di vendita deve essere unico. Considerando 6€ come costo di produzione di una maglietta, determina il **prezzo di vendita** a cui si otterrebbe il maggior  profitto totale (se il prezzo richiesto dal cliente è strettamente inferiore al prezzo di vendita, il cliente non acquisterà)


3. [1pt] Considera i **costi di consegna**: viene applicato un costo di 0.10€/km a capo per per ogni ordine sotto le 100 unità, mentre di 0.05€/km per ogni ordine di almeno 100 unità. Trova il nuovo prezzo di vendita che porti al maggior profitto totale e componi la lista degli ordini da eseguire, riordinata dall'ordine che porterebbe al profitto maggiore fino a quella che porterebbe al profitto minore.


4. [FACOLTATIVO per lode] Considera la curvatura terrestre per il calcolo della distanza per le consegne (riscrivi la funzione "Distance")


Utilizza il seguente codice come punto di partenza, riempiendo gli spazi dove manca codice!


Per qualsiasi domanda contattare Jean Paul G. Baroni +393315969286

In [69]:
# 1. Registra gli ordini in una lista di classi Order

GCS_Piazza_Leo = [45.478104, 9.227040] # Coordinate di Piazza Leonardo da Vinci, Milano
production_cost = 6 # Costo fisso di produzione della maglietta
num_orders = 420 # Numero di ordini nella cartella "data"

# Costo del trasporto
cost_less_100 = 0.10 #se l'ordine è minore di 100 unità
cost_at_least_100 = 0.05 # se l'ordine è almeno di 100 unità

orders = [] # Lista di oggetti che descrivono gli ordini ricevuti

# Classe che descrive ciascun ordine ricevuto
class Order:
    
    # Costruttore della classe (prende in input il frammento di email salvato nei .txt)
    def __init__(self, email_fragment):
        
        # Separa le righe del frammento e le salva nella lista lines
        lines = email_fragment.split("\n")
        
        # Salva la taglia della maglietta
        self.size = lines[1].split(" ")[-1]
        
        # Salva la quantità di magliette richieste
        self.quantity = int(lines[1].split(" ")[-5])
        
        # Salva la posizione di consegna dell'ordine come due elementi float di una lista
        self.GCS = [float(lines[2].split(" ")[-1].split(",")[0][1:]),float(lines[2].split(" ")[-1].split(",")[1][:-1])]
        
        # Salva il nome del cliente (ultima riga del txt)
        self.customer = lines[-1]
        
        # Salva il prezzo richiesto dal cliente (bid price)
        self.bid_price = lines[3].split(" ")[-1]
        self.bid_price = float(self.bid_price[:-1])
    
    # Calcola il pagamento che questo cliente effettuerebbe dato un prezzo di vendita
    def Payment(self, ask_price):
        return float(self.quantity)*ask_price
    
    # Calcola il profitto (ricavi - costi) per un ordine a questo cliente dato un prezzo di vendita
    def Profit(self, ask_price, cost_per_km):
        
        if (ask_price > self.bid_price):
            return 0
        
        quant = self.quantity
        
        if cost_per_km == False:
            return float(quant)*(ask_price - production_cost)
        
        if quant < 100:
            cost_per_km = cost_less_100
        else:
            cost_per_km = cost_at_least_100
        
        return quant*(ask_price - production_cost) - quant*self.Distance()*cost_per_km
        
    
    # Calcola una distanza approssimata in km tra piazza Leonardo da Vinci (punto di partenza delle consegne) e la destinazione
    def Distance(self, starting_GCS = GCS_Piazza_Leo):
        return float((((starting_GCS[0] - self.GCS[0])*111)**2+((starting_GCS[1] - self.GCS[1])*78)**2)**0.5)
    
    # Stampa i dettagli dell'oggetto
    def Details(self):
        print(self.customer,"ha ordinato",self.quantity,"magliette taglia",self.size,"a "+str(self.bid_price)+"€ all'indirizzo",self.GCS)
    
# Crea gli oggetti Order e li registra nella lista orders
for i in range(num_orders):
    
    file = open('Data/' + str(i+1)+'.txt','r')
    orders.append(Order(file.read()))
    
    file.close()

In [59]:
# 2. [1pt] Ogni cliente ha richiesto un prezzo diverso, ma il prezzo di vendita deve essere unico. Considerando 6€ 
# come costo di produzione di una maglietta, determina il **prezzo di vendita** a cui si otterrebbe il maggior 
# profitto totale (se il prezzo richiesto dal cliente è strettamente inferiore al prezzo di vendita, il cliente non acquisterà)

# Determina i profitti per tutti i possibili prezzi di vendita, riferendosi ai prezzi richiesti dai clienti 
possible_profits = []
for ordine in orders:
    price_candidate = ordine.bid_price
    guad = 0
    for ordine2 in orders:
        guad += ordine2.Profit(price_candidate,False)
    possible_profits.append(guad)
#_ = [print(pp) for pp in possible_profits]

# Determina il profitto maggiore
max_profit = max(possible_profits)

# Determina il prezzo migliore
best_index = possible_profits.index(max(possible_profits))     
best_price = orders[best_index].bid_price
print("Il miglior prezzo è €"+str(best_price))

# Trova gli ordini da eseguire dato il miglior prezzo
orders_to_go = [ord1 for ord1 in orders if ord1.Profit(best_price,0) > 0]
print("Si eseguiranno", len(orders_to_go),"ordini")

Il miglior prezzo è €13.22
Si eseguiranno 195 ordini


In [70]:
# 3. [1pt] Considera i **costi di consegna**: viene applicato un costo di 0.10€/km a capo per per ogni ordine sotto le 100 unità,
#mentre di 0.05€/km per ogni ordine di almeno 100 unità. Trova il nuovo prezzo di vendita che porti al maggior profitto totale e
#componi la lista degli ordini da eseguire, riordinata dall'ordine che porterebbe al profitto maggiore fino a quella che
#porterebbe al profitto minore.

cost_less_100 = 0.10
cost_at_least_100 = 0.05

# Determina i profitti per tutti i possibili prezzi di vendita, riferendosi ai prezzi richiesti dai clienti 
possible_profits = []
for ordine in orders:
    price_candidate = ordine.bid_price
    guad = 0
    for ordine2 in orders:
        guad += ordine2.Profit(price_candidate,True)
    possible_profits.append(guad)
#_ = [print(pp) for pp in possible_profits]

# Determina il profitto maggiore
max_profit = max(possible_profits)

# Determina il prezzo migliore
best_index = possible_profits.index(max(possible_profits))     
best_price = orders[best_index].bid_price
print("Il miglior prezzo è €"+str(best_price))

# Trova gli ordini da eseguire dato il miglior prezzo
orders_to_go = [ord1 for ord1 in orders if ord1.Profit(best_price,True) > 0]
print("Si eseguiranno", len(orders_to_go),"ordini")

######## SCRIVI CODICE ########
orders_to_go.sort(key = lambda x: x.Profit(best_price,True), reverse = True)
######## SCRIVI CODICE ########
print("\nOrdini da eseguire:")
for order in orders_to_go[0:10]:
    print (order.Details(),"portando un profitto di €",str(order.Profit(best_price,True)))

#_ = [otg.Details() for otg in orders_to_go]

Il miglior prezzo è €16.77
Si eseguiranno 76 ordini

Ordini da eseguire:
Gianmarco Fedeli Grissini ha ordinato 441 magliette taglia M a 16.77€ all'indirizzo [45.196751, 9.768684]
None portando un profitto di € 3591.107940790473
Filippo Fedeli Tentazioni ha ordinato 363 magliette taglia XXL a 19.2€ all'indirizzo [45.318257, 9.186061]
None portando un profitto di € 3582.290424584894
Davide Baroni Frumento ha ordinato 431 magliette taglia S a 19.62€ all'indirizzo [45.755771, 8.645121]
None portando un profitto di € 3459.530952944854
Carlo Augusto Fedeli Mascherine ha ordinato 357 magliette taglia XS a 17.5€ all'indirizzo [45.250094, 8.771933]
None portando un profitto di € 3066.686578014791
Maurizio Baroni Azienda Agricola ha ordinato 360 magliette taglia M a 17.72€ all'indirizzo [45.069648, 9.03768]
None portando un profitto di € 3018.8914902340734
Gianmarco Bressan Company ha ordinato 491 magliette taglia XS a 19.79€ all'indirizzo [45.661732, 10.428504]
None portando un profitto di € 29